In [35]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')
import env
import itertools

In [2]:
# function to deal with parsing one entry in our log data
def parse_log_entry(entry):
    parts = entry.split()
    output = {}
    output['ip'] = parts[0]
    output['timestamp'] = parts[3][1:].replace(':', ' ', 1)
    output['request_method'] = parts[5][1:]
    output['request_path'] = parts[6]
    output['http_version'] = parts[7][:-1]
    output['status_code'] = parts[8]
    output['size'] = int(parts[9])
    output['user_agent'] = ' '.join(parts[11:]).replace('"', '')
    return pd.Series(output)

url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/logs'
df = pd.read_sql('SELECT * FROM api_access', url)
# df = pd.concat([df.entry, df.entry.apply(parse_log_entry)], axis=1)
df = df.entry.apply(parse_log_entry)
df.head()

,ip,timestamp,request_method,request_path,http_version,status_code,size,user_agent
0,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/sales?page=81,HTTP/1.1,200,512495,python-requests/2.21.0
1,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/items,HTTP/1.1,200,3561,python-requests/2.21.0
2,97.105.19.58,16/Apr/2019 19:34:44,GET,/api/v1/sales?page=82,HTTP/1.1,200,510103,python-requests/2.21.0
3,97.105.19.58,16/Apr/2019 19:34:46,GET,/api/v1/sales?page=83,HTTP/1.1,200,510003,python-requests/2.21.0
4,97.105.19.58,16/Apr/2019 19:34:48,GET,/api/v1/sales?page=84,HTTP/1.1,200,511963,python-requests/2.21.0


In [12]:
df['timestamp'] = pd.to_datetime(df.timestamp)

In [22]:
df['timestamp_hour'] = df.timestamp.apply(lambda row: row.hour)

In [61]:
df['timestamp_day'] = df.timestamp.apply(lambda row: row.weekday())
df['timestamp_day_hour'] = df.timestamp.apply(lambda row: str(row.weekday()) + ', ' + str(row.hour))

In [62]:
for col in [col for col in df.columns if col != 'timestamp']:
    print('-' * 10, col.upper(), '-' * 10)
    print(df[col].value_counts(normalize=True))
    print()

---------- IP ----------
97.105.19.58      0.858595
173.173.113.51    0.075784
72.181.113.170    0.043867
72.181.105.81     0.017604
24.26.242.9       0.001503
68.201.219.223    0.001503
70.121.214.34     0.000143
52.87.230.102     0.000143
35.175.171.137    0.000143
54.145.52.184     0.000072
3.92.201.136      0.000072
35.174.209.2      0.000072
34.229.70.250     0.000072
54.172.14.223     0.000072
34.207.64.242     0.000072
3.88.129.158      0.000072
52.91.30.150      0.000072
52.90.165.200     0.000072
45.23.250.16      0.000072
Name: ip, dtype: float64

---------- REQUEST_METHOD ----------
GET    1.0
Name: request_method, dtype: float64

---------- REQUEST_PATH ----------
/api/v1/sales?page=2                 0.050737
/api/v1/items                        0.033205
/api/v1/items?page=2                 0.020824
/api/v1/items?page=3                 0.015672
/api/v1/stores                       0.011593
                                       ...   
/api/v1/items?page=0                 0.

In [110]:
proba_given_ip = {}
cols = ['status_code', 'user_agent', 'timestamp_day_hour']

for col in cols:
    
    proba_given_ip[col] = (
                           df.groupby('ip')[col]
                             .value_counts(normalize=True)
                             .rename('probability')
                             .reset_index()
                           )
    
    proba_given_ip[col]['n_requests'] = (
                                         df.groupby('ip')[col]
                                           .value_counts()
                                           .rename('n_requests')
                                           .reset_index()
                                           .n_requests
                                         )

In [111]:
proba_df = proba_given_ip['status_code']
proba_df[proba_df.probability < 1]

,ip,status_code,probability,n_requests
17,72.181.113.170,200,0.996737,611
18,72.181.113.170,499,0.003263,2
19,97.105.19.58,200,0.998833,11984
20,97.105.19.58,499,0.001167,14


In [112]:
proba_df = proba_given_ip['user_agent']
proba_df[proba_df.probability < 1]

,ip,user_agent,probability,n_requests
9,52.87.230.102,Slackbot 1.0 (+https://api.slack.com/robots),0.500000,1
10,52.87.230.102,Slackbot-LinkExpanding 1.0 (+https://api.slack...,0.500000,1
17,72.181.105.81,python-requests/2.21.0,0.934959,230
18,72.181.105.81,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; ...,0.032520,8
19,72.181.105.81,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,0.032520,8
20,72.181.113.170,python-requests/2.21.0,0.996737,611
21,72.181.113.170,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3...,0.003263,2
22,97.105.19.58,python-requests/2.21.0,0.838306,10058
23,97.105.19.58,python-requests/2.20.1,0.159277,1911
24,97.105.19.58,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,0.002167,26


In [113]:
proba_df = proba_given_ip['timestamp_day_hour']
proba_df[proba_df.probability < 1].sort_values(by=['ip', 'timestamp_day_hour'])

,ip,timestamp_day_hour,probability,n_requests
0,173.173.113.51,"2, 1",0.318225,337
4,173.173.113.51,"2, 2",0.039660,42
2,173.173.113.51,"2, 3",0.211520,224
3,173.173.113.51,"2, 4",0.197356,209
1,173.173.113.51,"2, 5",0.233239,247
6,24.26.242.9,"2, 5",0.285714,6
5,24.26.242.9,"2, 6",0.714286,15
14,52.87.230.102,"1, 21",0.500000,1
15,52.87.230.102,"2, 2",0.500000,1
21,68.201.219.223,"2, 1",0.476190,10
